In [1]:
import praw
import pandas as pd
import json 
import time

In [2]:
# Reddit API credentials

reddit = praw.Reddit(client_id="*****",               # your client id
                     client_secret= "*****",  # your client secret
                     user_agent= "*****")                          # your user agent

In [3]:
# Subreddit to scrape

subreddit = reddit.subreddit('depression')

In [4]:
# Define a function to recursively fetch replies for a comment

def fetch_replies(comment, limit=5):
    replies_data = []
    for reply in comment.replies[:limit]:  # Limit the number of replies
        reply_data = {
            'comment_id': reply.id,
            'author': reply.author.name if reply.author else None,
            'timestamp': reply.created_utc,
            'body': reply.body,
            'replies': fetch_replies(reply, limit)  # Recursively fetch replies for each reply
        }
        replies_data.append(reply_data)
    return replies_data

In [5]:
# Define a function to fetch data with a delay between requests

def fetch_data_with_delay(limit=None, delay=2, reply_limit=5):
    posts_data = []
    for post in subreddit.top(limit=limit):
        # Fetch post data
        post_data = {
            'post_id': post.id,
            'title': post.title,
            'author': post.author.name if post.author else None,
            'timestamp': post.created_utc,
            'body': post.selftext,
            'score': post.score,
            'downs': post.downs,
            'total_comments': post.num_comments,
            'comments': []  
        }
        # Fetch comments for the post
        post.comments.replace_more(limit=10)
        for comment in post.comments:
            # Fetch comment data
            comment_data = {
                'comment_id': comment.id,
                'author': comment.author.name if comment.author else None,
                'timestamp': comment.created_utc,
                'body': comment.body,
                'replies': fetch_replies(comment, limit=reply_limit)  # Limit replies for the comment
            }
            post_data['comments'].append(comment_data)
        posts_data.append(post_data)
        time.sleep(delay)  # Add delay between requests
    return posts_data

In [8]:
# Fetch data with a delay of 2 seconds between requests and a limit of 5 replies per comment

data = fetch_data_with_delay(limit=None, delay=2, reply_limit=5)

In [9]:
# Save data as JSON
with open('top_depression.json', 'w') as f:
   json.dump(data, f, indent=2)